In [32]:
import keras
import numpy as np
from pond.tensor import NativeTensor, PrivateEncodedTensor, PublicEncodedTensor
from pond.nn import Dense
import time

## Conv2D Layer

Some small unit-tests for the Conv2D layer. run forward pass with mnist image, and backward pass with random noise

In [33]:

class Conv2DNaive():
        
    def __init__(self, fshape, strides=1, filter_init=lambda shp: np.random.normal(scale=0.1, size=shp)):
        """ 2 Dimensional convolutional layer
            fshape: tuple of rank 4
            strides: int with stride size
            filter init: lambda function with shape parameter
            Example: Conv2D((4, 4, 1, 20), strides=2, filter_init=lambda shp: np.random.normal(scale=0.01, size=shp))       
        """
        self.fshape = fshape
        self.strides = strides
        self.filter_init = filter_init
        self.cache = None
        self.initializer = None
        
    def initialize(self):
        self.filters = self.filter_init(self.fshape)

    def forward(self, x):
        # TODO: padding 
        s = (x.shape[1] - self.fshape[0]) // self.strides + 1
        self.initializer =  type(x)
        fmap = self.initializer(np.zeros((x.shape[0], s, s, self.fshape[-1])))
        
        for j in range(s):
            for i in range(s):
                fmap[:, j, i, :] = (x[:, j * self.strides:j * self.strides + self.fshape[0], i * self.strides:i * self.strides + self.fshape[1], :, np.newaxis] * self.filters).sum(axis=(1, 2, 3))
        self.cache = x
        return fmap
    
    def backward(self, d_y, learning_rate):
        x = self.cache
        # compute gradients for internal parameters and update
        d_weights = self.get_grad(x, d_y)
        self.filters = (d_weights * learning_rate).neg() + self.filters
        # compute and return external gradient
        d_x = self.backwarded_error(d_y)
        return d_x
    
    def backwarded_error(self, layer_err):
        bfmap_shape = (layer_err.shape[1] - 1) * self.strides + self.fshape[0]
        backwarded_fmap = self.initializer(np.zeros((layer_err.shape[0], bfmap_shape, bfmap_shape, self.fshape[-2])))
        s = (backwarded_fmap.shape[1] - self.fshape[0]) // self.strides + 1
        for j in range(s):
            for i in range(s):
                backwarded_fmap[:, j * self.strides:j  * self.strides + self.fshape[0], i * self.strides:i * self.strides + self.fshape[1]] += (self.filters[np.newaxis, ...] * layer_err[:, j:j+1, i:i+1, np.newaxis, :]).sum(axis=4)
        return backwarded_fmap

    def get_grad(self, x, layer_err):
        total_layer_err = layer_err.sum(axis=(0, 1, 2))
        filters_err = self.initializer(np.zeros(self.fshape))
        s = (x.shape[1] - self.fshape[0]) // self.strides + 1
        summed_x = x.sum(axis=0)
        for j in range(s):
            for i in range(s):
                filters_err += summed_x[j  * self.strides:j * self.strides + self.fshape[0], i * self.strides:i * self.strides + self.fshape[1], :, np.newaxis]
        return filters_err * total_layer_err



In [34]:
# read data
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
image_batch = x_train[0:64,:,:,np.newaxis] / 255.0

## NativeTensor

In [35]:
# forward pass
conv_layer = Conv2DNaive((4, 4, 1, 20), strides=2, filter_init=lambda shp: np.random.normal(scale=0.01, size=shp))
conv_layer.initialize()
start = time.time()
output = conv_layer.forward(NativeTensor(image_batch))
#backward pass (with random update)
delta = NativeTensor(np.random.normal(size=output.shape))
lr = 0.01
_ = conv_layer.backward(d_y=delta, learning_rate=lr)
print(time.time()-start)

0.06033015251159668


## PublicEncodedTensor

In [36]:
# forward pass
conv_layer = Conv2DNaive((4, 4, 1, 20), strides=2, filter_init=lambda shp: np.random.normal(scale=0.01, size=shp))
conv_layer.initialize()
start = time.time()
output = conv_layer.forward(PublicEncodedTensor(image_batch))
#backward pass (with random update)
delta = PublicEncodedTensor(np.random.normal(size=output.shape))
lr = 0.01
_ = conv_layer.backward(d_y=delta, learning_rate=lr)
print(time.time()-start)

6.182796239852905


## PrivateEncodedTensor

In [37]:
# forward pass
conv_layer = Conv2DNaive((4, 4, 1, 20), strides=2, filter_init=lambda shp: np.random.normal(scale=0.01, size=shp))
conv_layer.initialize()
start = time.time()
output = conv_layer.forward(PrivateEncodedTensor(image_batch))
#backward pass (with random update)
delta = PrivateEncodedTensor(np.random.normal(size=output.shape))
lr = 0.01
_ = conv_layer.backward(d_y=delta, learning_rate=lr)
print(time.time()-start)

67.20014929771423


## Conv2D optimized

In [38]:
from im2col import im2col_indices, col2im_indices
# read data in NCHW instead of NHWC for the optimized version
image_batch = x_train[0:64,np.newaxis, :,:] / 255.0

In [39]:
class Conv2D():
    def __init__(self, fshape, strides=1, padding=0, filter_init=lambda shp: np.random.normal(scale=0.1, size=shp),
                 channels_first=True):
        """ 2 Dimensional convolutional layer, expects NCHW data format
            fshape: tuple of rank 4
            strides: int with stride size
            filter init: lambda function with shape parameter
            Example: Conv2D((4, 4, 1, 20), strides=2, filter_init=lambda shp: np.random.normal(scale=0.01,
            size=shp))
        """
        self.fshape = fshape
        self.strides = strides
        self.padding = padding
        self.filter_init = filter_init
        self.cache = None
        self.cached_input_shape = None
        self.initializer = None
        assert channels_first

    def initialize(self):
        self.filters = NativeTensor(self.filter_init(self.fshape))

    def forward(self, x):
        # TODO: padding
        self.initializer = type(x)

        # shapes, assuming NCHW
        h_filter, w_filter, d_filters, n_filters = self.filters.shape
        n_x, d_x, h_x, w_x = x.shape


        h_out = int((h_x - h_filter + 2 * self.padding) / self.strides + 1)
        w_out = int((w_x - w_filter + 2 * self.padding) / self.strides + 1)

        X_col = im2col_indices(x, field_height=h_filter, field_width=w_filter,
                               padding=self.padding, stride=self.strides)
        W_col = self.filters.reshape(n_filters, -1)
        # multiplication
        out = W_col.dot(X_col)
        out = out.reshape(self.fshape[3], h_out, w_out, n_x)

        out = out.reshape(n_filters, h_out, w_out, n_x)
        out = out.transpose(3, 0, 1, 2)
        self.cache = X_col
        self.cached_input_shape = x.shape

        return out

    def backward(self, d_y, learning_rate):
        X_col = self.cache
        h_filter, w_filter, d_filter, n_filter = self.filters.shape

        dout_reshaped = d_y.transpose(1, 2, 3, 0).reshape(n_filter, -1)
        dw = dout_reshaped.dot(X_col.transpose())
        dw = dw.reshape(self.filters.shape)
        self.filters = (dw * learning_rate).neg() + self.filters

        W_reshape = self.filters.reshape(n_filter, -1)
        dx_col = W_reshape.transpose().dot(dout_reshaped)
        dx = col2im_indices(dx_col, self.cached_input_shape, self.initializer,field_height=h_filter,
                            field_width=w_filter, padding=self.padding, stride=self.strides)

        return dx


In [40]:
# forward pass
conv_layer = Conv2D((4, 4, 1, 20), strides=2, filter_init=lambda shp: np.random.normal(scale=0.01, size=shp))
conv_layer.initialize()
start = time.time()
output = conv_layer.forward(PrivateEncodedTensor(image_batch))
#backward pass (with random update)
delta = PrivateEncodedTensor(np.random.normal(size=output.shape))
lr = 0.01
_ = conv_layer.backward(d_y=delta, learning_rate=lr)
print(time.time()-start)

14.625490665435791
